# Read dataset

In [29]:
import pandas as pd
import numpy as np
from ml_functions import *

df = pd.read_csv('toy_dataset.csv', index_col=0)
df.shape

(4046, 26)

Intial dataset size.

In [30]:
df[['rxn_a','rxn_b','rxn_c']] = df['id'].str.split(',', expand=True)

In [31]:
def count_deletions(x):

    return len(x.split(','))

no_delitions = []
for row in range(len(df)):

    num = count_deletions(df['id'].iloc[row])
    no_delitions.append(num)

df['no_deletions'] = no_delitions
df.head()

,r1,r3,r2,r4,r5,r6,r7,r8,r9,r10,r11,r12,r13,outputK,r14,r15,r16,InputA,InputI,OutputX,OutputN,id,growth,strain,infeasible,optimal,rxn_a,rxn_b,rxn_c,no_deletions
0,0.0,0.1,-0.1,-0.2,1.2,1.0,1.0,1.0,1.0,-6.938894e-17,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,r1,3.0,1,1,0,r1,None,None,1
1,1.0,0.0,1.0,2.0,1.0,3.0,3.0,3.0,1.0,0.000000e+00,1.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,1.0,9.0,1.0,r3,9.0,1,0,1,r3,None,None,1
2,1.0,1.0,0.0,0.0,3.0,3.0,3.0,3.0,1.0,0.000000e+00,1.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,1.0,9.0,1.0,r2,9.0,1,0,1,r2,None,None,1
3,1.0,1.0,0.0,0.0,3.0,3.0,3.0,3.0,1.0,0.000000e+00,1.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,1.0,9.0,1.0,r4,9.0,1,0,1,r4,None,None,1
4,1.0,0.1,0.9,1.8,1.2,3.0,3.0,3.0,1.0,0.000000e+00,1.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,1.0,9.0,1.0,r5,9.0,1,1,0,r5,None,None,1


### Store reactions names

In [32]:
# Store all reactions names
gem_rxn = df[df['no_deletions'] == 1]['id'].unique()

# Some statistical study

In [33]:
print('Infeasible', round(df['optimal'].value_counts()[0]/len(df) * 100,2), '% of the dataset')
print('Optimal', round(df['optimal'].value_counts()[1]/len(df) * 100,2), '% of the dataset')

Infeasible 69.25 % of the dataset
Optimal 30.75 % of the dataset


In [34]:
# Subset of df containing only infeasible rows
df_inf = df[df['optimal'] == 0]

Οι αντιδράσεις που φαίνονται στο επόμενο κελί __ΔΕΝ__ οδήγησαν __ποτέ__ σε _infeasible_ status όταν κόπηκαν.

Μια λογική είναι να κοπούν οι παρακάτω αντιδράσεις. Εξαίρεση, οι αντιδράσεις που αφορούν input/output.

In [35]:
set(df_inf[df_inf['no_deletions']==1]['id'].unique()) ^ set(gem_rxn)

{'InputA', 'InputI', 'OutputN', 'r10', 'r14', 'r15', 'r2', 'r3', 'r4'}

Οι παρακάτω αντιδράσεις εάν κοπούν οδηγούν σε **infeasible** κατάσταση. Επομένως, ΔΕ θα πρέπει να περιλαμβάνονται στα deletions. Οι αντιδράσεις αυτές, δηλαδή, θα πρέπει να **προστατευτούν** από τα reaction deletions.

In [36]:
critical_rxn = df_inf[df_inf['no_deletions']==1]['id'].unique()
critical_rxn

array(['r1', 'r5', 'r6', 'r7', 'r8', 'r9', 'r11', 'r12', 'r13', 'outputK',
       'r16', 'OutputX'], dtype=object)

Παρακάτω είναι οι αντιδράσεις που θα μπορούσαν να κοπούν.

In [37]:
rxn_deletable = list(set(df_inf[df_inf['no_deletions']==1]['id'].unique()) ^ set(gem_rxn))
rxn_deletable

['r10', 'r2', 'InputI', 'r4', 'r14', 'r3', 'r15', 'InputA', 'OutputN']

In [38]:
index_del_rxn_a = df.loc[df['rxn_a'].isin(critical_rxn)].index
df.drop(index_del_rxn_a, inplace=True)

index_del_rxn_b = df.loc[df['rxn_b'].isin(critical_rxn)].index
df.drop(index_del_rxn_b, inplace=True)

index_del_rxn_c = df.loc[df['rxn_c'].isin(critical_rxn)].index
df.drop(index_del_rxn_c, inplace=True)

In [39]:
df.shape

(438, 30)

### Find and Delete Duplicated Rows 

In [40]:
index_a = df[df['rxn_a'] == df['rxn_b']].index
df.drop(index_a, axis=0, inplace=True)

index_b = df[df['rxn_a'] == df['rxn_c']].index
df.drop(index_b, axis=0, inplace=True)

index_c = df[df['rxn_b'] == df['rxn_c']].index
df.drop(index_c, axis=0, inplace=True)

df.shape

(258, 30)

In [41]:
df.head()

,r1,r3,r2,r4,r5,r6,r7,r8,r9,r10,r11,r12,r13,outputK,r14,r15,r16,InputA,InputI,OutputX,OutputN,id,growth,strain,infeasible,optimal,rxn_a,rxn_b,rxn_c,no_deletions
1,1.0,0.0,1.0,2.0,1.0,3.0,3.0,3.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,1.0,9.0,1.0,r3,9.0,1,0,1,r3,None,None,1
2,1.0,1.0,0.0,0.0,3.0,3.0,3.0,3.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,1.0,9.0,1.0,r2,9.0,1,0,1,r2,None,None,1
3,1.0,1.0,0.0,0.0,3.0,3.0,3.0,3.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,1.0,9.0,1.0,r4,9.0,1,0,1,r4,None,None,1
9,1.0,0.1,0.9,1.8,1.2,3.0,3.0,3.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,1.0,9.0,1.0,r10,9.0,1,0,1,r10,None,None,1
14,1.0,0.1,0.9,1.8,1.2,3.0,3.0,3.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,1.0,9.0,1.0,r14,9.0,1,0,1,r14,None,None,1


In [42]:
deletions = pd.DataFrame()

for rxn in rxn_deletable:
    deletions[rxn] = np.nan
    
deletions['id'] = df['id']
deletions.tail()

for row in range(deletions.shape[0]):

    deleted_rxn = deletions.iloc[row]['id']
    
    for col in deletions.columns:
        if(col in deleted_rxn): deletions[col].iloc[row] = 1

deletions.fillna(0, inplace=True)

new_names = []
for col in deletions.columns:
    new_names.append('del_'+col)
    
deletions.columns = new_names
deletions.drop('del_id', axis=1, inplace=True)

df = pd.concat([df, deletions], axis=1)
df.head()

/tmp/ipykernel_10714/2394415776.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if(col in deleted_rxn): deletions[col].iloc[row] = 1


,r1,r3,r2,r4,r5,r6,r7,r8,r9,r10,r11,r12,r13,outputK,r14,r15,r16,InputA,InputI,OutputX,OutputN,id,growth,strain,infeasible,optimal,rxn_a,rxn_b,rxn_c,no_deletions,del_r10,del_r2,del_InputI,del_r4,del_r14,del_r3,del_r15,del_InputA,del_OutputN
1,1.0,0.0,1.0,2.0,1.0,3.0,3.0,3.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,1.0,9.0,1.0,r3,9.0,1,0,1,r3,None,None,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,1.0,0.0,0.0,3.0,3.0,3.0,3.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,1.0,9.0,1.0,r2,9.0,1,0,1,r2,None,None,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,3.0,3.0,3.0,3.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,1.0,9.0,1.0,r4,9.0,1,0,1,r4,None,None,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,1.0,0.1,0.9,1.8,1.2,3.0,3.0,3.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,1.0,9.0,1.0,r10,9.0,1,0,1,r10,None,None,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,1.0,0.1,0.9,1.8,1.2,3.0,3.0,3.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,1.0,9.0,1.0,r14,9.0,1,0,1,r14,None,None,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [43]:
pd.set_option('display.max_columns', None)
df.tail()

,r1,r3,r2,r4,r5,r6,r7,r8,r9,r10,r11,r12,r13,outputK,r14,r15,r16,InputA,InputI,OutputX,OutputN,id,growth,strain,infeasible,optimal,rxn_a,rxn_b,rxn_c,no_deletions,del_r10,del_r2,del_InputI,del_r4,del_r14,del_r3,del_r15,del_InputA,del_OutputN
3986,1.0,1.0,-5.551115e-16,-4.440892e-16,2.0,2.000000e+00,2.000000e+00,2.000000e+00,0.0,5.921189e-16,5.921189e-16,2.220446e-16,2.220446e-16,1.332268e-15,0.000000e+00,1.479114e-31,1.000000e+00,1.0,0.0,6.000000e+00,0.0,"r14,InputI,OutputN",6.000000e+00,2,0,1,r14,InputI,OutputN,3,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
4002,0.0,0.1,-1.000000e-01,-2.000000e-01,0.2,2.312965e-17,2.312965e-17,2.775558e-17,0.0,-1.850372e-17,-1.850372e-17,-1.387779e-17,-1.387779e-17,-2.775558e-17,3.081488e-33,0.000000e+00,-1.387779e-17,0.0,0.0,4.163336e-17,0.0,"r15,InputA,InputI",4.163336e-17,2,0,1,r15,InputA,InputI,3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4004,0.0,0.1,-1.000000e-01,-2.000000e-01,0.2,2.312965e-17,2.312965e-17,2.775558e-17,0.0,-1.850372e-17,-1.850372e-17,-1.387779e-17,-1.387779e-17,-2.775558e-17,3.081488e-33,0.000000e+00,-1.387779e-17,0.0,0.0,4.163336e-17,0.0,"r15,InputA,OutputN",4.163336e-17,2,0,1,r15,InputA,OutputN,3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
4007,1.0,1.0,-1.110223e-16,8.881784e-16,2.0,2.000000e+00,2.000000e+00,2.000000e+00,0.0,1.480297e-16,1.480297e-16,-4.440892e-16,-4.440892e-16,1.332268e-15,-1.314768e-31,0.000000e+00,1.000000e+00,1.0,0.0,6.000000e+00,0.0,"r15,InputI,OutputN",6.000000e+00,2,0,1,r15,InputI,OutputN,3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
4032,0.0,1.0,-1.000000e+00,-2.000000e+00,2.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.0,"InputA,InputI,OutputN",0.000000e+00,2,0,1,InputA,InputI,OutputN,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0


In [45]:
df.shape

(258, 39)

# Further Study dataset

In [14]:
print('Infeasible', round(df['optimal'].value_counts()[0]/len(df) * 100,2), '% of the dataset')
print('Optimal', round(df['optimal'].value_counts()[1]/len(df) * 100,2), '% of the dataset')

Infeasible 31.01 % of the dataset
Optimal 68.99 % of the dataset


In [ ]:
print('Infeasible', round(df['optimal'].value_counts()[0], 2))
print('Optimal', round(df['optimal'].value_counts()[1],2))

In [ ]:
pd.DataFrame(df[['growth', 'OutputX']].describe()).T

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.histplot(data=df, x="growth", bins=20)
plt.show()

## Negative growth

In [ ]:
df[df['growth'] < 0]

11 rows of __negative__ growth

### Check Linear Correlations

##### Point Biserial Correlation

In [ ]:
pearsons = df.drop(columns=critical_rxn).corr(numeric_only=True)['optimal'].sort_values(ascending=False)
spearman = df.drop(columns=critical_rxn).corr(numeric_only=True, method='spearman')['optimal'].sort_values(ascending=False)

corr_matrix = pd.DataFrame({'pearsons':pearsons, 'spearman':spearman})
corr_matrix.sort_values(by='pearsons')

# AI / ML Ideas

## Create _training_ and _test_ sets and _data normalization_

Create a `df` keeping only _non critical_ reactions.

In [ ]:
df.drop(columns=critical_rxn, inplace=True)
df.head()

In [ ]:
# Create dummy variables for rxn_a, rxn_b

dummy_rxna = pd.get_dummies(df['rxn_a'], prefix='rxna')
dummy_rxnb = pd.get_dummies(df['rxn_b'], prefix='rxnb')
dummy_rxnc = pd.get_dummies(df['rxn_c'], prefix='rxnc')

# dummies = pd.concat([dummy_rxna, dummy_rxnb], axis=1)

# dummies.head(2)

In [ ]:
deleted_rxn = pd.DataFrame()

for rxna in dummy_rxna.columns:
    for rxnb in dummy_rxnb.columns:
        for rxnc in dummy_rxnc.columns:
                
            if((rxna.split("_")[1] == rxnb.split("_")[1]) 
               or (rxna.split("_")[1] == rxnc.split("_")[1])
               or (rxnb.split("_")[1] == rxnc.split("_")[1])):
                deleted_rxn[rxna] = (dummy_rxna[rxna] + dummy_rxnb[rxnb] + dummy_rxnc[rxnc])

In [ ]:
for rxna in dummy_rxna.columns:
    for rxnb in dummy_rxnb.columns:
                    
        if((rxna.split("_")[1] == rxnb.split("_")[1]) or (rxna.split("_")[1] == rxnc.split("_")[1])):
            deleted_rxn[rxna] = (dummy_rxna[rxna] + dummy_rxnb[rxnb])

In [ ]:
for rxna in dummy_rxna.columns:
    for rxnc in dummy_rxnc.columns:
        
        if(rxna.split("_")[1] == rxnc.split("_")[1]):
            deleted_rxn[rxna] = deleted_rxn[rxna] + dummy_rxnc[rxnc]

col = 0
for rxnb in dummy_rxnb.columns:
    for rxnc in dummy_rxnc.columns:
        
        if((rxnb.split("_")[1] == deleted_rxn.columns[col].split("_")[1]) and (rxnb.split("_")[1] == rxnc.split("_")[1])):
            c = deleted_rxn.columns[col]
            deleted_rxn[c] = deleted_rxn[c] + dummy_rxnc[rxnc]

    
    col = col + 1

In [ ]:
pd.set_option('display.max_rows', 260)
deleted_rxn

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### Define **target** variable

We need to better define the target that we want to predict!

(The most simple and obvious answer is to predit **optimal** column)

In [ ]:
X = df.drop(['optimal', 'infeasible', 'growth', 'rxn_a', 'rxn_b', 'rxn_c'], axis=1)
y = df['infeasible']

print(X.shape)
print(y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30, stratify=y, random_state=42)

In [ ]:
# normalize data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
print(y_train.value_counts(normalize=True))
print(y_test.value_counts(normalize=True))
print(df['optimal'].value_counts(normalize=True))

### SVM Classifier

In [ ]:
svc_model, svc_scores = train_svm(X_train_scaled, y_train)

In [ ]:
validation_classification(svc_model, X_test_scaled, y_test)

### XGBoost Classifier

In [ ]:
xgb_model, xgb_scores = xgb_classifier(X_train_scaled, y_train)

In [ ]:
validation_classification(xgb_model, X_test_scaled, y_test)

#### Usage of UnderSampling and Oversampling

In [ ]:
import imblearn
from imblearn.under_sampling import TomekLinks
from collections import Counter

In [ ]:
# define the undersampling method
undersample = TomekLinks()
X_train_under, y_train_under = undersample.fit_resample(X_train_scaled, y_train)

In [ ]:
print( Counter(y_train) )

print( Counter (y_train_under) )

# Feature Importance to detect reactions for knock-out

In [ ]:
import shap

explainer = shap.TreeExplainer(xgb_model, X_train_scaled, feature_names=X_train.columns)
shap_values = explainer(X_test_scaled)

In [ ]:
shap.plots.beeswarm(shap_values, max_display=X_train.shape[1])

In [ ]:
shap.plots.bar(shap_values, max_display=X_test.shape[1])

In [ ]:
importances = xgb_model.feature_importances_
indices = np.argsort(importances)
features = X_train.columns
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='g', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

# Artificial Neural Networks

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [ ]:
n_inputs = len(X_train.columns)

In [ ]:
model = Sequential()
model.add(Dense(120, input_dim=n_inputs, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = [
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ])

history = model.fit(X_train_scaled, y_train, epochs=10, batch_size=5)

In [ ]:
validation_classification(model, X_test_scaled, y_test)

In [ ]:
import shap


explainer = shap.DeepExplainer(model, X_train_scaled)

In [ ]:
shap_values = explainer.shap_values(X_test_scaled)

In [ ]:
shap.summary_plot(shap_values[0], plot_type = 'bar', feature_names = X_test.columns, max_display=X_test.shape[1])

In [ ]:
deletions = pd.DataFrame()

for rxn in rxn_deletable:
    deletions[rxn] = np.nan
    
deletions['id'] = df['id']
deletions.tail()

for row in range(deletions.shape[0]):

    deleted_rxn = deletions.iloc[row]['id']
    
    for col in deletions.columns:
        if(col in deleted_rxn): deletions[col].iloc[row] = 1

deletions.fillna(0, inplace=True)

deletions.tail()